# Monte Carlo simulation for confidence interval

In [4]:
library(dslabs)
library(tidyverse)
library(Rmisc)

Loading required package: lattice

Loading required package: plyr

------------------------------------------------------------------------------

You have loaded plyr after dplyr - this is likely to cause problems.
If you need functions from both plyr and dplyr, please load plyr first, then dplyr:
library(plyr); library(dplyr)

------------------------------------------------------------------------------


Attaching package: 'plyr'


The following objects are masked from 'package:dplyr':

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


The following object is masked from 'package:purrr':

    compact




In [2]:
heights %>% glimpse()

Rows: 1,050
Columns: 2
$ sex    <fct> Male, Male, Male, Male, Male, Female, Female, Female, Female...
$ height <dbl> 75, 70, 68, 74, 61, 65, 66, 62, 66, 67, 72, 72, 69, 68, 69, ...


In [3]:
# the population is the height of man
population <- heights %>% filter(sex == 'Male') %>% pull(height)

In [7]:
# sample size
n <- 50
# number of simulation
n_simulation <- 10000
confidence_level <- .95
# calculate confidence interval for 1 simulation
conduct <- function() {
    sample_data <- sample(population, size = n, replace = T)
    CI(sample_data, ci = confidence_level)
}

confidence_intervals <- 1:n_simulation %>% map_dfr(~conduct())

confidence_intervals %>% slice_head(n = 5)

upper,mean,lower
<dbl>,<dbl>,<dbl>
71.09466,70.20860,69.32254
70.68331,69.70511,68.72691
70.80103,69.67759,68.55415
70.95732,70.00443,69.05154
70.32938,69.49073,68.65207


We expect the 95% simulated confidence interval will contains the true mean height of male

In [9]:
m <- mean(population) # true mean
confidence_intervals %>% summarize(p = mean(lower <= m & m <= upper) * 100)

p
<dbl>
95.19


# Monte Carlo simulation for Bayesian statistics

<pre>P(having disease) = 0.00025  
P(positive test|having disease) = 0.99  
P(negative test|not having disease) = 0.99  
P(having disease|positive test) = <span style = 'color:red'>?</span></pre>

In [64]:
# probability of a person having disease
p = 0.00025
# probability of positive test given a person having disease
p_have_disease_test = 0.99
# probability of negative test given a person not having disease
p_not_disease_test = 0.99
n = 100000 # sample size
# 0 indicates not having disease, 1 indicate having disease
observation_sample <- sample(c('disease', 'healthy'), size = n, replace = T, prob = c(p,1 - p))

In [65]:
n_disease <- sum(observation_sample == 'disease') # the number of people having disease in our sample
n_disease

[1] 25

In [66]:
test_result <- character(n)
# test result for people who do not have disease
test_result[observation_sample == 'healthy'] = sample(c('-', '+'), size = n - n_disease, replace = T, prob = c(p_not_disease_test, 1 - p_not_disease_test))
# test result for people who have disease
test_result[observation_sample == 'disease'] = sample(c('-', '+'), size = n_disease, replace = T, prob = c(1 - p_have_disease_test, p_have_disease_test))

In [67]:
test_summary <- table(observation_sample, test_result)
test_summary

                  test_result
observation_sample     +     -
           disease    25     0
           healthy   998 98977

In [68]:
# given positive test, what is the probability a person actually have disease? (in theory, 2%)
test_summary['disease', '+'] / sum(test_summary[, '+']) * 100

[1] 2.443793